
# Customer Sentiment

In [ ]:
#!pip install OpenAI
#!pip install langchain
#!pip install langchain_community
#!pip install Cohere
#!pip install langchain-openai langchain-cohere python-dotenv

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
from google.colab import userdata

In [ ]:
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key


In [ ]:

import os, json
import pandas as pd



In [ ]:

from langchain_openai import ChatOpenAI


In [ ]:

CSV_PATH = "https://github.com/giridhar276/genai/raw/refs/heads/main/datasets/Bank_Customer_conversations.csv"
MODEL = "gpt-4o-mini"
TEMPERATURE = 0
TIMEOUT = 60

OUTPUT_PATH = CSV_PATH.replace(".csv", "_with_sentiment_oneprompt.csv")



In [ ]:

PROMPT = (
    "You are a sentiment classifier. Return ONE LINE of JSON with keys "
    '\"label\", \"confidence\", \"reason\". '
    'Label must be one of "positive", "neutral", "negative". '
    "Confidence is a number in [0,1]. Reason is <= 15 words. "
    "No extra text before/after JSON.\n\n"
    "Classify the customer text:\n"
    '\"\"\"{text}\"\"\"'
)


In [ ]:

df = pd.read_csv(CSV_PATH)
df = df[["customer_text"]].copy()
df["customer_text"] = df["customer_text"].astype(str).str.strip()

print("Rows:", len(df))
df.head(3)


In [ ]:

llm = ChatOpenAI(model=MODEL, temperature=TEMPERATURE, timeout=TIMEOUT)

def classify_one(text: str):
    resp = llm.invoke(PROMPT.format(text=text))
    return json.loads(resp.content)  # assumes valid one-line JSON


In [ ]:

labels, confs, reasons = [], [], []

for txt in df["customer_text"].tolist():
    obj = classify_one(txt)
    labels.append(obj["label"])
    confs.append(obj["confidence"])
    reasons.append(obj["reason"])


In [ ]:

df["sentiment_label"] = labels
df["sentiment_confidence"] = confs
df["sentiment_reason"] = reasons



In [ ]:

df.to_csv("finalsentiment.csv", index=False)
print(f"Saved: {OUTPUT_PATH}")



In [ ]:
df.head()

In [ ]:
print(df["sentiment_label"].value_counts())


In [ ]:
df.shape